### Reading xml

In [66]:
#import xml.etree.ElementTree as ET
import lxml.etree as ET
from xml.dom import minidom

import numpy as np
from PIL import Image
from skimage import measure
Image.MAX_IMAGE_PIXELS = None

In [111]:
XML_PATH = '/BrainSeg/data/box_Ab/NA3777-02_AB.xml'

In [5]:
tree = ET.parse(XML_PATH)
root = tree.getroot()

In [9]:
root.attrib

{'MicronsPerPixel': '0.502700'}

In [12]:
for child in root:
    print(child.tag, child.attrib)
    for childchild in child:
        print(childchild.tag, childchild.attrib)

Annotation {'Id': '1', 'Name': '', 'ReadOnly': '0', 'NameReadOnly': '0', 'LineColorReadOnly': '0', 'Incremental': '0', 'Type': '4', 'LineColor': '65280', 'Visible': '1', 'Selected': '1', 'MarkupImagePath': '', 'MacroName': ''}
Attributes {}
Regions {}
Plots {}


### Writing xml

In [98]:
SAVE_PATH = '/BrainSeg/NA3777-02_AB_new.xml'

In [99]:
def get_regions_node(parent_node):
    regions_node = ET.SubElement(parent_node, 'Regions')
    headers_node = ET.SubElement(regions_node, 'RegionAttributeHeaders')
    ET.SubElement(headers_node, 'AttributeHeader', attrib={'Id': "9999", 'Name': "Region", 'ColumnWidth': "-1"})
    ET.SubElement(headers_node, 'AttributeHeader', attrib={'Id': "9997", 'Name': "Length", 'ColumnWidth': "-1"})
    ET.SubElement(headers_node, 'AttributeHeader', attrib={'Id': "9996", 'Name': "Area", 'ColumnWidth': "-1"})
    ET.SubElement(headers_node, 'AttributeHeader', attrib={'Id': "9998", 'Name': "Text", 'ColumnWidth': "-1"})
    return regions_node

In [109]:
# Create base structure
# ET.SubElement(parent, tag, attrib={}, **extra)
root = ET.Element('Annotations')
GM_node = ET.SubElement(root, 'Annotation', attrib={'Id': "1", 'Name': "Gray Matter"})
WM_node = ET.SubElement(root, 'Annotation', attrib={'Id': "2", 'Name': "White Matter"})

GM_regions_node = get_regions_node(GM_node)
WM_regions_node = get_regions_node(WM_node)

ET.SubElement(GM_node, 'Plots')
ET.SubElement(WM_node, 'Plots')

<Element Plots at 0x7f60aa2dcd48>

In [114]:
tree = ET.ElementTree(root)
tree.write(SAVE_PATH, pretty_print=True)

### Writing boundary to xml

In [108]:
BOUNDARY_PATH = '/BrainSeg/data/outputs/UNet_Zero_Pad_2019O-512-BSCCE_cp-003-0.8061_postproc_method6_boundary/NA3777-02_AB.png'
MASK_PATH = '/BrainSeg/data/outputs/UNet_Zero_Pad_2019O-512-BSCCE_cp-003-0.8061_postproc_method6/NA4626-02_AB.png'

In [64]:
# boundary_img_arr = np.array(Image.open(BOUNDARY_PATH))

In [110]:
def build_boundary_xml(mask_arr, regions_node, downsample_rate=50):
    contours = measure.find_contours(mask_arr, 0, fully_connected='high')
    
    for i, contour in enumerate(contours):
        region_node = ET.SubElement(regions_node, 'Region', attrib={'Id': str(i+1), 'DisplayId': str(i+1)})
        vertices_node = ET.SubElement(region_node, 'Vertices')
        for y, x in contour[::downsample_rate]: 
            ET.SubElement(vertices_node, 'Vertex', attrib={'X': str(int(x)),'Y': str(int(y)), 'Z': "0"})

In [112]:
mask_arr = np.array(Image.open(MASK_PATH))

In [113]:
build_boundary_xml(mask_arr==1, GM_regions_node)
build_boundary_xml(mask_arr==2, WM_regions_node)